In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/WUSTL/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9998827239867143,
        0.9600649800293866, array([1, 1, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9996984331086939,
        0.9305545272093338, array([3, 3, 3, ..., 3, 3, 3])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.9984796002563319,
        0.7518383136072314, array([5, 5, 5, ..., 5, 5, 5])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9998701586995765,
        0.939002641264322, array([1, 1, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9971518682487759,
        0.8894642081911899, array([3, 3, 3, ..., 3, 3, 3])],
       ['FGSM', 0.3, 'Def

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
import numpy as np
from sklearn.cluster import KMeans

def stratified_cluster_split(X, y, test_size=0.5, random_state=42):
    X_sub_list = []
    y_sub_list = []
    X_rest_list = []
    y_rest_list = []

    classes = np.unique(y)

    rng = np.random.RandomState(seed=random_state)

    for c in classes:
        X_c = X[y == c]
        y_c = y[y == c]

        if len(X_c) < 2:
            X_rest_list.append(X_c)
            y_rest_list.append(y_c)
            continue

        kmeans = KMeans(n_clusters=2, random_state=random_state)
        cluster_labels = kmeans.fit_predict(X_c)

        cluster_0_idx = np.where(cluster_labels == 0)[0]
        cluster_1_idx = np.where(cluster_labels == 1)[0]

        if len(cluster_0_idx) <= len(cluster_1_idx):
            base_idx = cluster_0_idx
        else:
            base_idx = cluster_1_idx

        n_select = max(1, min(int(len(X_c) * test_size), len(X_c) - 1))

        if len(base_idx) >= n_select:
            selected_idx = rng.choice(base_idx, size=n_select, replace=False)
        else:
            extra_needed = n_select - len(base_idx)
            other_idx = np.setdiff1d(np.arange(len(X_c)), base_idx)
            extra_idx = rng.choice(other_idx, size=extra_needed, replace=False)
            selected_idx = np.concatenate([base_idx, extra_idx])

        rest_idx = np.setdiff1d(np.arange(len(X_c)), selected_idx)

        X_sub_list.append(X_c[selected_idx])
        y_sub_list.append(y_c[selected_idx])
        X_rest_list.append(X_c[rest_idx])
        y_rest_list.append(y_c[rest_idx])

    X_sub = np.concatenate(X_sub_list, axis=0)
    y_sub = np.concatenate(y_sub_list, axis=0)
    X_rest = np.concatenate(X_rest_list, axis=0)
    y_rest = np.concatenate(y_rest_list, axis=0)

    return X_rest, X_sub, y_rest, y_sub

_, X_10, _, y_10 = stratified_cluster_split(x_test_adv_0_1, y_label_0_1, test_size=0.1, random_state=42)



In [6]:
X_10.shape

(214873, 42)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb10 = XGBClassifier()
xgb10.fit(X_10, y_10)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf10 = RandomForestClassifier()
rf10.fit(X_10, y_10)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt10 = DecisionTreeClassifier()
# dt10.fit(X_10, y_10)

xgb
RF


RandomForestClassifier()

In [8]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

In [9]:


base_path_template = "/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = xgb10.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Cluster/WUSTL_Input10/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Cluster/WUSTL_Input10/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[1 3 4 5] [    40 228789   3161   6763]
(40, 42) (40,)
Save 1 to baseline_Def1.npy
(228789, 42) (228789,)
Save 3 to baseline_Def3.npy
(3161, 42) (3161,)
Save 4 to baseline_Def4.npy
(6763, 42) (6763,)
Save 5 to baseline_Def5.npy
Execution Time: 1.171303 seconds
(716259,)
[ 1  3  4  5  6 10] [ 28485 271344      3   1718 413998    711]
(28485, 42) (28485,)
Save 1 to BIM_Def1.npy
(271344, 42) (271344,)
Save 3 to BIM_Def3.npy
(3, 42) (3,)
Save 4 to BIM_Def4.npy
(1718, 42) (1718,)
Save 5 to BIM_Def5.npy
(413998, 42) (413998,)
Save 6 to BIM_Def6.npy
(711, 42) (711,)
Save 10 to BIM_Def10.npy
Execution Time: 5.533456 seconds
(716259,)
[ 1  3  5  6  7  9 10] [ 13770 252247   9177 440865     64     16    120]
(13770, 42) (13770,)
Save 1 to FGSM_Def1.npy
(252247, 42) (252247,)
Save 3 to FGSM_Def3.npy
(9177, 42) (9177,)
Save 5 to FGSM_Def5.npy
(440865, 42) (440865,)
Save 6 to FGSM_Def6.npy
(64, 42) (64,)
Save 7 to FGSM_Def7.npy
(16, 42) (16,)
Save 9 to FGSM_Def9.npy
(120, 42) (120,)
Save 

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = rf10.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Cluster/WUSTL_Input10/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Cluster/WUSTL_Input10/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[3 4 5 8] [232287     57   6406      3]
(232287, 42) (232287,)
Save 3 to baseline_Def3.npy
(57, 42) (57,)
Save 4 to baseline_Def4.npy
(6406, 42) (6406,)
Save 5 to baseline_Def5.npy
(3, 42) (3,)
Save 8 to baseline_Def8.npy
Execution Time: 5.268790 seconds
(716259,)
[1 3 4 5 6] [205057 277852     19  22827 210504]
(205057, 42) (205057,)
Save 1 to BIM_Def1.npy
(277852, 42) (277852,)
Save 3 to BIM_Def3.npy
(19, 42) (19,)
Save 4 to BIM_Def4.npy
(22827, 42) (22827,)
Save 5 to BIM_Def5.npy
(210504, 42) (210504,)
Save 6 to BIM_Def6.npy
Execution Time: 17.132220 seconds
(716259,)
[1 3 5 6 9] [  9556 252676 230943 223071     13]
(9556, 42) (9556,)
Save 1 to FGSM_Def1.npy
(252676, 42) (252676,)
Save 3 to FGSM_Def3.npy
(230943, 42) (230943,)
Save 5 to FGSM_Def5.npy
(223071, 42) (223071,)
Save 6 to FGSM_Def6.npy
(13, 42) (13,)
Save 9 to FGSM_Def9.npy
Execution Time: 20.548403 seconds
(716259,)
[1 3 4 5 6] [205057 277852     19  22827 210504]
(205057, 42) (205057,)
Save 1 to PGD_Def1.npy
(

In [11]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

#     y_pred = dt10.predict(X)
#     print(y_pred.shape)
#     y_pred = label_encoder.inverse_transform(y_pred)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_Cluster/WUSTL_Input10/x_test_adv_{attack_name}_Def{class_num}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_Cluster/WUSTL_Input10/y_test_adv_{attack_name}_Def{class_num}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

In [12]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}10")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/{m_name}_Cluster/WUSTL_Input10/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [13]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input10_Cluster.csv")

In [14]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB10,40,0,228789,3161,6763,0,0,0,0,0,0,238753
1,XGB10,28485,0,271344,3,1718,413998,0,0,0,711,0,716259
2,XGB10,13770,0,252247,0,9177,440865,64,0,16,120,0,716259
3,XGB10,28485,0,271344,3,1718,413998,0,0,0,711,0,716259
4,XGB10,2368,0,25570,34,3448,682933,0,0,147,1759,0,716259
5,XGB10,41539,0,428951,451,5786,230908,81,0,56,8486,1,716259
6,XGB10,120,0,686364,9483,20292,0,0,0,0,0,0,716259
7,XGB10,55604,182,388145,10326,96417,0,1156,164081,348,0,0,716259
8,XGB10,952,0,273130,0,105,442071,1,0,0,0,0,716259
9,XGB10,78184,0,255077,0,687,381755,0,0,23,533,0,716259
